In [ ]:
import os
import sys

cwd0 = './config/'
sys.path.append(cwd0)

import visualID_Eng as vID
from visualID_Eng import color,fg,hl,bg
vID.init(cwd0)

import tools4pyPhysChem as t4pPC

#cancel the "last operation show-up" specific of Jupyter notebooks
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


# Prediction by an artificial neural network of the solubility of CO<sub>2</sub> in ionic liquids

<div class="intro">
    
<b>Reference</b>: 
Z. Song, H. Shi, X. Zhang & T. Zhou (**2020**), Prediction of CO<sub>2</sub> solubility in ionic liquids using machine learning methods, [<i>Chem. Eng. Sci.</i> <b>223</b>: 115752](https://www.doi.org/10.1016/j.ces.2020.115752) 
<br>
<p style="text-align: center"><img width="650px" src="./DS4B-CO2-images/AbstractANNCO2-SongEtal.png" style="margin-left:auto; margin-right:auto" id="img_AbstractSong"></p>
<br>
The main results are graphically reported below.
<br>
<p style="text-align: center"><img width="900px" src="./DS4B-CO2-images/ANNCO2-SongEtal-Results.png" style="margin-left:auto; margin-right:auto" id="img_ResultsSong"></p>
<br>

**The goal of this tutorial is to reproduce the ANN part of this article, which is an example of application of a DFF**

<img src="./DS4B-Slides/pngs/ZooNN/DFF-C.png" alt="DFF" width="180" style="display: block; margin: 0 auto; align:center" id="DFF"/>
</div>


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import os,sys
from IPython.display import display

<a id="data-read"></a>
## Basic data analysis and visualization

<div class="intro">

The [pandas library](https://pandas.pydata.org/docs/index.html) provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language. It is recommended to first have a look at the [10mn short introduction to pandas](https://pandas.pydata.org/docs/user_guide/10min.html).
</div>

### Read the database

<div class="intro">

It is made of 57 columns:
- the name of the IL, under its abbreviated form. For example [BMIM][PF6] is for [1-Butyl-3-methylimidazolium hexafluorophosphate](https://en.wikipedia.org/wiki/1-Butyl-3-methylimidazolium_hexafluorophosphate)
- the abbreviated name of the cation and of the anion in two separated columns
- the CO<sub>2</sub> solubility (x_CO2)
- the temperature and pressure under which the solubility was measured 
- the 51 remaining columns are molecular descriptors. The authors chose:
    - group contribution (GC) descriptors for the cations, *i.e.* the occurrences of functional groups in the molecule
    - a one hot encoding scheme for anions, e.g. [1,0,...,0] (with 27 zeros) for [BF4], which is the first anion of this database, that contains ILs made of 28 different anions
</div>

In [ ]:
dataCO2f='DS4B-CO2-data'+'/'+'dataCO2.csv'
dataCO2=pd.read_csv(dataCO2f,sep=";",header=0)
display(dataCO2)
# describe() generates descriptive statistics
display(dataCO2.describe().style.format("{0:.2f}"))


In [ ]:
t4pPC.centerTitle("Unique ILs")
nIL = dataCO2["IL"].nunique()
print(f"There are {nIL} different ionic liquids in this database")
print(dataCO2["IL"].unique())

t4pPC.centerTitle("Unique cations")
nCat = dataCO2["cation"].nunique()
print(f"There are {nCat} different cations in this database")
print(dataCO2["cation"].unique())

t4pPC.centerTitle("Unique anions")
nAn = dataCO2["anion"].nunique()
print(f"There are {nAn} different anions in this database")
print(dataCO2["anion"].unique())

### How do the cations an anions look like?

<div class="intro">

Their SMILES coding is available in a separated csv file. `rdkit` will now be used to see the 2D structure of these species. The SMILES must first be converted into rdkit `mol` objects. 
</div>

In [ ]:
import rdkit
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
print(f"{color.BOLD}{color.BLUE}Version of rdkit : {rdkit.__version__}{color.OFF}")
fSMILESILs='DS4B-CO2-data'+'/'+'dataCO2_SMILES_ILsList.csv'
SMILESILs=pd.read_csv(fSMILESILs,sep="\t",header=0)
display(SMILESILs)

In [ ]:
ILcation=SMILESILs[SMILESILs['Cation or Anion'] == 'Cation']
ILanion=SMILESILs[SMILESILs['Cation or Anion'] == 'Anion']
PandasTools.AddMoleculeColumnToFrame(ILcation, smilesCol='SMILES')
PandasTools.AddMoleculeColumnToFrame(ILanion, smilesCol='SMILES')
pd.options.display.max_rows = 5
display(ILcation)
display(ILanion)

#### Display cations on a grid

In [ ]:
IPythonConsole.drawOptions.bondLineWidth=2
#FrameToGridImage: Draw grid image of mols in pandas DataFrame
imgCations = PandasTools.FrameToGridImage(ILcation, molsPerRow=6, legendsCol='Abbreviation', useSVG=True, subImgSize=(300, 200), maxMols=ILcation.shape[0])
display(imgCations)
with open('DS4B-CO2-images/GridCations.svg', 'w') as file: file.write(imgCations.data)

#### Display anions on a grid

In [ ]:
IPythonConsole.drawOptions.bondLineWidth=2
#FrameToGridImage: Draw grid image of mols in pandas DataFrame
imgAnions = PandasTools.FrameToGridImage(ILanion, molsPerRow=6, legendsCol='Abbreviation', useSVG=True, subImgSize=(300, 200), maxMols=ILcation.shape[0])
display(imgAnions)
with open('DS4B-CO2-images/GridAnions.svg', 'w') as file: file.write(imgAnions.data)

### Data distribution?

In [ ]:
vID.chrono_start()
dataCO2.hist(figsize=(24,24))
print(dataCO2[["x_CO2", "[CH3]"]].groupby("[CH3]").count())
dataCO2.hist("[CH3]",figsize=(10,6),facecolor='g')
plt.show()
vID.chrono_show()

## Prediction of the solubility of CO<sub>2</sub> by an ANN: <br>a textbook case of work that does not quite meet the standards of the field 

### ML scheme of the reference article

<div class="intro">
    
<p style="text-align: center"><img width="800px" src="./DS4B-CO2-images/ANN-CO2Song_etal.png" style="margin-left:auto; margin-right:auto" id="img_Song"></p>

**Methodological details (excerpts from the article):**
- The MATLAB tansig and purelin transfer functions are employed in the hidden and output layers, respectively
- The experimental data are divided into a training set (8093 points, 80% of the data) to build the model and a test set of the remaining 2023 data points to evaluate the predictive capability of the obtained model
- <span style="color:red"><b>!!!</b> Instead of performing random selection, we employ a hybrid artificial-random strategy to decompose the dataset. Specifically, the data points consisting of the least frequently used groups are equally divided into five folders <b>!!!</b></span> 

</div>
<br>
<div class="rqT" title="Warning!">

This "*hybrid artificial-random strategy*" is not a valid strategy. The recommended K-fold cross validation, applied to the 3rd model, cannot be applied here!

**Let's observe now the weird behaviour of this model**
</div>

### Useful functions

In [ ]:
def plotModel(model, model_id, MODELS_PATH=".", fig_extension="png", resolution=300,
              show_shapes=True, show_layer_names=True, show_layer_activations=True):
    '''
    saves a model plot to a given subfolder, that will be created if it does not exist
    input:
        - model = keras model
        - model_id = name of the model file, without an extension
        - MODELS_PATH = pathway to the folder. Default: "."
        - model_extension = model type. Default: png
    '''
    from tensorflow.keras.utils import plot_model
    from matplotlib.pyplot import plot
    path = MODELS_PATH / f"{model_id}.{fig_extension}"
    plot_model(model, to_file=path, dpi=resolution, show_shapes=show_shapes, 
               show_layer_names=show_layer_names, show_layer_activations=show_layer_activations)
    img = plt.imread(path)
    plt.imshow(img)
    plt.axis("off")
    plt.show()
    return


### Model 1: assesment of the reproducibility of the simulation of Song *et al.*<a id="model1"></a>

<div class = "intro">
    
Errors calculated between the actual and predicted x_CO2 values (noted $Y$ and $\hat{Y}$ in equations below)  
MAE =  Mean Absolute Error 
$$\mathrm{MAE}=\frac{1}{n}\sum_{i}\left|\hat{Y}(x_{i})-Y(x_{i})\right|$$

MSE = Mean Squared Error  
$$\mathrm{MSE}=\frac{1}{n}\sum_{i}\left(\hat{Y}(x_{i})-Y(x_{i})\right)^{2}$$

</div>

#### Split the data into train and test sets

In [ ]:
#######################################################################################
# separation of the data set into two subsets: (1) training of the ANN & (2) test of the ANN
# library used: pandas
data_train = dataCO2.sample(frac=0.8, axis=0)
data_test  = dataCO2.drop(data_train.index)

x_train = data_train.drop(['IL','cation','anion','x_CO2'],axis=1)
y_train = data_train['x_CO2']
x_test  = data_test.drop(['IL','cation','anion','x_CO2'],axis=1)
y_test  = data_test['x_CO2']

display(x_train,y_train)
display(x_test,y_test)

#### Define the ANN

In [ ]:
#######################################################################################
# ANN: 1 input layer (53 neurons) / 1 hidden layer (7 neurons with the reLu activation function) / 1 output layer (1 neuron with no activation function) 
# library used: keras

def defANN(shape,acthL):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name='iLayer'))
    model.add(keras.layers.Dense(7, activation=acthL, name='hLayer'))
    model.add(keras.layers.Dense(1, name='oLayer'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

acthL='tanh'
#acthL='relu'
if 'ANNmodel' in globals():
    del ANNmodel
if 'ANNhistory' in globals():
    del ANNhistory
ANNmodel=defANN( (53,), acthL )
ANNmodel.summary()

#### Plot the model

In [ ]:
from pathlib import Path

MODELS_PATH = Path() / "DS4B-CO2-SavedModels"
MODELS_PATH.mkdir(parents=True, exist_ok=True)
plotModel(ANNmodel, "Model1_CO2", MODELS_PATH=MODELS_PATH)

#### Train the ANN

In [ ]:
#######################################################################################
# optimization of the ANN
# library used: keras

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
vID.chrono_start()
ANNhistory = ANNmodel.fit(x_train,
                    y_train,
                    epochs          = 300,
                    batch_size      = 20,
                    verbose         = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[es])
vID.chrono_show()

#######################################################################################
# accuracy of the ANN?
# library used: numpy

ytrain_hat=ANNmodel.predict(x_train)
ytest_hat=ANNmodel.predict(x_test)
# numpy.ravel() returns a contiguous flattened array
diffyt = ytrain_hat - y_train.to_numpy()
diffyp = ytest_hat - y_test.to_numpy()

print()
print("xCO2(predicted) - xCO2(actual)")
print(
      "Train.","mean: ", np.mean(diffyt),
      "   std: ", np.std(diffyt),
      "   MAE: ", np.average(abs(diffyt)),
      "    R2: ", np.corrcoef(y_train.to_numpy(),ytrain_hat.flatten())[0,1]
     )
print(
      "Test.","mean: ", np.mean(diffyp),
      "   std: ", np.std(diffyp),
      "   MAE: ", np.average(abs(diffyp)),
      "    R2: ", np.corrcoef(y_test.to_numpy(),ytest_hat.flatten())[0,1]
     )


<div class="rq"><span style="color:red; font-weight:bold">The MAE and R2 parameters seem rather good (<i>ca.</i> 0.07 and 0.92), but this prediction is actually very bad, as you will see by jumping to the <a href="#model-evaluation">Model Evaluation</a> section!</span>
<br>
<br>  
Unless you were lucky (an element of chance is involved in the learning process), the predicted <i>vs.</i> actual CO<sub>2</sub> solubility probably looks like this: 
<br>
<p style="text-align: center"><img width="550px" src="./DS4B-CO2-images/predictionCO2_woStd-saved.png" style="margin-left:auto; margin-right:auto" id="img_predCO2woStd"></p>
<b>It differs a lot from the results of the reference article, reported in the introduction of this notebook.</b>
<br><br>
<span style="color:red; font-weight:bold">Origin of this weird behaviour?</span>
<br>
Probably the absence of <b>standardization</b> of the data. This scattering is by the way reminiscent the variation of the solubility as a function of <i>T</i> or <i>p</i>, plotted below. Let's check that in the next script.
</div>

In [ ]:
cm = 1/2.54
plt.rcParams["figure.figsize"] = (20*cm,12*cm) #graphic size
fig1=dataCO2.plot(x="T (K)", y=["x_CO2"],linestyle='', marker='o',fontsize=14)
fig2=dataCO2.plot(x="P (bar)", y=["x_CO2"],linestyle='', marker='x',fontsize=14)

#### Save the Keras ANN model for later use

In [ ]:
ANNmodel.save('./DS4B-CO2-SavedModels/ANNmodel-CO2-basic.keras')

<a id="model2"></a>
## Model 2 = standardization of *T* and *p* only

<div class="rqT" title="What is standardization?">

**Standardization** is the process of translating and scaling our features so that they are all *distributed around a mean of zero* with a *standard deviation of one*.
    
Because the Temperature and pressure in the CO<sub>2</sub> dataset are on totally different scales, we need to scale them, **without changing their relative distribution**
    
We want to standardize our data so that the *covariances* are easily comparable for each pair of features. If we don’t do it, features with larger ranges of numbers will have higher covariances, which could be an artificial feature.
</div>

<br>
<div class="intro">

This new model  is an improvement of model 1, but without standardizing the GC descriptors which covariances are small. <br>
</div>
<br>

### Split into train and test sets

In [ ]:
# Resets all state generated by Keras
tf.keras.backend.clear_session()

In [ ]:
#######################################################################################
# separation of the data set into two subsets: (1) training of the ANN & (2) test of the ANN
# library used: pandas
data_train = dataCO2.sample(frac=0.8, axis=0)
data_test  = dataCO2.drop(data_train.index)

x_train = data_train.drop(['IL','cation','anion','x_CO2'],axis=1)
y_train = data_train['x_CO2']
x_test  = data_test.drop(['IL','cation','anion','x_CO2'],axis=1)
y_test  = data_test['x_CO2']

#######################################################################################

# Train
x_train = data_train.drop(['IL','cation','anion','x_CO2'],axis=1)
xIL_train = x_train.drop(['T (K)','P (bar)'],axis=1)
xTP_train = data_train[['T (K)','P (bar)']]
y_train = data_train['x_CO2']
x_trainTmp = x_train.copy() # for plotting purpose

# Test
x_test  = data_test.drop(['IL','cation','anion','x_CO2'],axis=1)
xIL_test = x_test.drop(['T (K)','P (bar)'],axis=1)
xTP_test = data_test[['T (K)','P (bar)']]
y_test  = data_test['x_CO2']
x_testTmp = x_test.copy() # for plotting purpose

print(f"{bg.DARKREDB}Training set before normalization:{bg.OFF}")
display(x_train)
display(x_train.describe().style.format("{0:.2f}").set_caption("Training set before normalization:"))
print(f"{bg.DARKREDB}Test set before normalization:{bg.OFF}")
display(x_test)
display(x_test.describe().style.format("{0:.2f}").set_caption("Test set before normalization:"))

### Scale independently T & P

In [ ]:
# using scikit-learn
from sklearn import preprocessing

print(f"{bg.LIGHTBLUEB}Training set (T,P) after normalization:{bg.OFF}")
scalerTP = preprocessing.StandardScaler()
scalerTP.fit(xTP_train.values)
xTP_train_scaled = scalerTP.transform(xTP_train.values) #returns a numpy array
xTP_train = pd.DataFrame(xTP_train_scaled, index=xTP_train.index, columns=xTP_train.columns)
display(xTP_train)
display(xTP_train.describe().style.format("{0:.2f}"))

print(f"{bg.LIGHTBLUEB}Test set (T,P) after normalization:{bg.OFF}")
xTP_test_scaled = scalerTP.transform(xTP_test.values) #returns a numpy array
xTP_test = pd.DataFrame(xTP_test_scaled, index=xTP_test.index, columns=xTP_test.columns)
display(xTP_test)
display(xTP_test.describe().style.format("{0:.2f}"))


### Merge the (xTP_train & xIL_train) dfs & (xTP_test & xIL_test) dfs

In [ ]:
x_train = pd.concat([xTP_train,xIL_train], axis=1)
x_test = pd.concat([xTP_test,xIL_test], axis=1)
display(x_train.describe().style.format("{0:.2f}"))
display(x_test.describe().style.format("{0:.2f}"))

In [ ]:
#######################################################################################
# graphic representation of the previous tables
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

nbins=30
fig = plt.figure(figsize=(18, 12))
plt.rcParams["font.size"] = (14) #font size
plt.tick_params(axis='x',labelsize = 16)
plt.tick_params(axis='y',labelsize = 12)
gs = gridspec.GridSpec(2, 4, height_ratios=[1, 1], width_ratios=[1, 1, 1, 1])

ax00=plt.subplot(gs[0,0], title="Tr. set before standardization")
fig00=sns.histplot(data=x_trainTmp,x="T (K)", bins=nbins, kde=True, color="#1f77b4", alpha=1.0)
ax02=plt.subplot(gs[0,2], title="Tr. set before standardization")
fig02=sns.histplot(data=x_trainTmp,x="P (bar)", bins=nbins, kde=True, color="#3e95ff", alpha=1.0)
ax01=plt.subplot(gs[0,1], title="Tr. set after standardization")
fig01=sns.histplot(data=x_train,x="T (K)", bins=nbins, kde=True, color="#1f77b4", alpha=1.0)
ax03=plt.subplot(gs[0,3], title="Tr. set after standardization")
fig11=sns.histplot(data=x_train,x="P (bar)", bins=nbins, kde=True, color="#3e95ff", alpha=1.0)

ax10=plt.subplot(gs[1,0], title="Ts. set before standardization")
fig10=sns.histplot(data=x_testTmp,x="T (K)", bins=nbins, kde=True, color="#1f77b4", alpha=1.0)
ax12=plt.subplot(gs[1,2], title="Ts. set before standardization")
fig12=sns.histplot(data=x_testTmp,x="P (bar)", bins=nbins, kde=True, color="#3e95ff", alpha=1.0)
ax11=plt.subplot(gs[1,1], title="Ts. set after standardization")
fig11=sns.histplot(data=x_test,x="T (K)", bins=nbins, kde=True, color="#1f77b4", alpha=1.0)
ax13=plt.subplot(gs[1,3], title="Ts. set after standardization")
fig13=sns.histplot(data=x_test,x="P (bar)", bins=nbins, kde=True, color="#3e95ff", alpha=1.0)


plt.savefig('./DS4B-CO2-images/Standardization_CO2data.png',dpi=300,bbox_inches='tight')

### Delete the scaled & Tmp dfs

In [ ]:
del x_testTmp, x_trainTmp, xTP_test_scaled, xTP_train_scaled

### Now define and train the ANN

In [ ]:
#######################################################################################
# ANN: 1 input layer (53 neurons) / 2 hidden layers (20 and 7 neurons) / 1 output layer (1 neuron) 
# library used: keras

def defANN(shape,acthL):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name='iLayer'))
    model.add(keras.layers.Dense(7, activation=acthL, name='hLayer'))
    model.add(keras.layers.Dense(1, name='oLayer'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

acthL='tanh'
#acthL='relu'
if 'ANNmodel' in globals():
    del ANNmodel
if 'ANNhistory' in globals():
    del ANNhistory
ANNmodel=defANN( (53,), acthL )
ANNmodel.summary()

In [ ]:
#######################################################################################
# optimization of the ANN
# library used: keras

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

vID.chrono_start()
ANNhistory = ANNmodel.fit(x_train,
                    y_train,
                    epochs          = 300,
                    batch_size      = 20,
                    verbose         = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[es])
vID.chrono_show()

#######################################################################################
# accuracy of the ANN?
# library used: numpy

ytrain_hat=ANNmodel.predict(x_train)
ytest_hat=ANNmodel.predict(x_test)
# numpy.ravel() returns a contiguous flattened array
diffyt = ytrain_hat.flatten() - y_train.to_numpy()
diffyp = ytest_hat.flatten() - y_test.to_numpy()

print()
print("xCO2(predicted) - xCO2(actual)")
print(
      "Train.","mean: ", np.mean(diffyt),
      "   std: ", np.std(diffyt),
      "   MAE: ", np.average(abs(diffyt)),
      "    R2: ", np.corrcoef(y_train.to_numpy(),ytrain_hat.flatten())[0,1]
     )
print(
      "Test.","mean: ", np.mean(diffyp),
      "   std: ", np.std(diffyp),
      "   MAE: ", np.average(abs(diffyp)),
      "    R2: ", np.corrcoef(y_test.to_numpy(),ytest_hat.flatten())[0,1]
     )


### Save the Keras ANN model and the scaler for later use

In [ ]:
# save the model
ANNmodel.save('./DS4B-CO2-SavedModels/ANNmodel-CO2-StandardizedB.keras')

# save the scalers
from pickle import dump
dump(scalerTP, open('./DS4B-CO2-SavedModels/scalerB_TP.pkl', 'wb'))

<div class="rqE">

Once completed, jump to the <a href="#model-evaluation">Model Evaluation</a> section

</div>

## Assessment of the stability of the ML model 2 by *K*-fold cross validation<a id="model4"></a>

<div class="rqE">Model 2 is now adapted to assess how the results of the ML generalizes</div>
<br>

<div class="intro">
The main idea behind cross-validation is that each sample in a dataset has the opportunity of being tested. K-fold cross-validation is a special case of cross-validation where we iterate over a dataset set <i>k</i> times. The dataset is split into <i>k</i> parts at each iteration: one part is used for validation, and the remaining <i>k</i>−1  parts are merged into a training subset for model evaluation. The figure below illustrates the process of 5-fold cross-validation
<p style="text-align: center"><img width="520px" src="DS4B-svg/KFoldCV.svg" style="margin-left:auto; margin-right:auto" id="img_5FoldCV"></p>
</div>

### Apply a 5-fold cross validation

In [ ]:
# Resets all state generated by Keras
tf.keras.backend.clear_session()

In [ ]:
#######################################################################################
# separation of the data set into two subsets: (1) training of the ANN & (2) test of the ANN
# library used: pandas
xdata = dataCO2.drop(['IL','cation','anion','x_CO2'],axis=1)
xdataIL = xdata.drop(['T (K)','P (bar)'],axis=1)
xdataTP = xdata[['T (K)','P (bar)']]
ydata = dataCO2['x_CO2']

#######################################################################################
# ANN: 1 input layer (53 neurons) / 2 hidden layers (20 and 7 neurons) / 1 output layer (1 neuron) 
# library used: keras

def defANN(shape,acthL):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name='iLayer'))
    model.add(keras.layers.Dense(7, activation=acthL, name='hLayer'))
    model.add(keras.layers.Dense(1, name='oLayer'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

acthL='tanh'

In [ ]:
#######################################################################################
# optimization of the ANN
# library used: keras and scikit learn for the KFold cross-validator
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
from sklearn.model_selection import KFold
from sklearn import preprocessing
from pickle import dump

VERBOSE = 1
EPOCHS = 300
BATCH_SIZE = 25
N_SPLIT = 5

vID.chrono_start()

# variables created to save at each iteration of the KFold process: the mean error, the standard deviation, MAE, R2
meantT=list()
stdtT=list()
MAEtT=list()
R2tT=list()
meanvT=list()
stdvT=list()
MAEvT=list()
R2vT=list()

kfold = KFold(n_splits=N_SPLIT,shuffle=True,random_state=42) # k-fold is here!
#print(list(kfold.split(x_train,y_train)))

j = 0 # Variable for keeping count of split we are executing
# The KFold cv provides train/test indices to split data in train/test sets
for train_idx, val_idx in list(kfold.split(xdata,ydata)):

    xIL_train_cv = xdataIL.iloc[train_idx]
    xIL_valid_cv = xdataIL.iloc[val_idx]
    xTP_train_cv = xdataTP.iloc[train_idx]
    xTP_valid_cv = xdataTP.iloc[val_idx]
    y_train_cv = ydata.iloc[train_idx]
    y_valid_cv = ydata.iloc[val_idx]
#    display(x_train_cv,x_valid_cv)
    scalerTP = preprocessing.StandardScaler()
    scalerTP.fit(xTP_train_cv.values)
    xTPt_scaled = scalerTP.transform(xTP_train_cv.values) #returns a numpy array
    xTPv_scaled = scalerTP.transform(xTP_valid_cv.values) #returns a numpy array
    xtmp_train_cv = pd.DataFrame(xTPt_scaled, index=xTP_train_cv.index, columns=xTP_train_cv.columns) #conversion of a numpy array into a pandas dataframe
    xtmp_valid_cv = pd.DataFrame(xTPv_scaled, index=xTP_valid_cv.index, columns=xTP_valid_cv.columns) #conversion of a numpy array into a pandas dataframe
    x_train_cv = pd.concat([xtmp_train_cv,xIL_train_cv], axis=1)
    x_valid_cv = pd.concat([xtmp_valid_cv,xIL_valid_cv], axis=1)
    del xTPt_scaled, xTPv_scaled, xtmp_train_cv, xtmp_valid_cv, xIL_train_cv, xIL_valid_cv
    del xTP_train_cv, xTP_valid_cv
#    display(x_train_cv.describe().style.format("{0:.2f}").set_caption("Training set after normalization (with scikit-learn):"))
#    display(x_valid_cv.describe().style.format("{0:.2f}").set_caption("Validation set after normalization (with scikit-learn):"))
    print(f"{color.BOLD}{color.RED}Fold {j}{color.OFF}")
    j+=1
    ANNmodel=defANN( (53,), acthL )
    ANNhistory = ANNmodel.fit(x_train_cv,
                        y_train_cv,
                        epochs          = EPOCHS,
                        batch_size      = BATCH_SIZE,
                        verbose         = VERBOSE,
                        validation_data = (x_valid_cv, y_valid_cv),
                        callbacks=[es])
    ytrain_hat=ANNmodel.predict(x_train_cv)
    yvalid_hat=ANNmodel.predict(x_valid_cv)
    # numpy.ravel() returns a contiguous flattened array
    diffyt = ytrain_hat.flatten() - y_train_cv.to_numpy()
    diffyv = yvalid_hat.flatten() - y_valid_cv.to_numpy()

    print()
    print("xCO2(predicted) - xCO2(actual)")
    print(
          "Train.","mean: ", np.mean(diffyt),
          "   std: ", np.std(diffyt),
          "   MAE: ", np.average(abs(diffyt)),
          "    R2: ", np.corrcoef(y_train_cv.to_numpy(),ytrain_hat.flatten())[0,1]
         )
    print(
          "Test.","mean: ", np.mean(diffyv),
          "   std: ", np.std(diffyv),
          "   MAE: ", np.average(abs(diffyv)),
          "    R2: ", np.corrcoef(y_valid_cv.to_numpy(),yvalid_hat.flatten())[0,1]
         )
    meantT.append(np.mean(diffyt))
    meanvT.append(np.mean(diffyv))
    stdtT.append(np.std(diffyt))
    stdvT.append(np.std(diffyv))
    MAEtT.append(np.average(abs(diffyt)))
    MAEvT.append(np.average(abs(diffyv)))
    R2tT.append(np.corrcoef(y_train_cv.to_numpy(),ytrain_hat.flatten())[0,1])
    R2vT.append(np.corrcoef(y_valid_cv.to_numpy(),yvalid_hat.flatten())[0,1])
    
    # Save the Keras ANN model for later use    
    ModelName = './DS4B-CO2-SavedModels/ANNmodel-CO2-StandardizedB-' + str(j-1) + '.keras'
    ANNmodel.save(ModelName)
    # Scaler also saved!!!!!!!!!!!!!!
    ScalerName = './DS4B-CO2-SavedModels/scalerB-' + str(j-1) + '.pkl'
    dump(scalerTP, open(ScalerName, 'wb'))
    
vID.chrono_show()

#######################################################################################
# accuracy of the ANN?
# library used: numpy
print(f"{color.BOLD}average MAE of the training set:{color.OFF}   {np.mean(MAEtT):.2f} +/- {np.std(MAEtT):.2f}")
print(f"{color.BOLD}average MAE of the validation set:{color.OFF} {np.mean(MAEvT):.2f} +/- {np.std(MAEvT):.2f}")

### Visual assessment of the 5-fold cross validation

In [ ]:
figCV, axCV = plt.subplots(1, 1)
figCV.set_size_inches(12,5)
axCV.errorbar(x=np.arange(len(meantT)), y=meantT, yerr=MAEtT, label='training sets', fmt='o-', capsize=10)
axCV.errorbar(x=np.arange(len(meanvT))+0.1, y=meanvT, yerr=MAEvT, label='validation sets', fmt='o-', capsize=10)
axCV.legend(loc='lower left', shadow=True, fontsize='14')
axCV.set_xlabel('cross-validation k-values ',fontdict={'fontsize':16})
axCV.set_ylabel('$\hat{y}-y_{\mathrm{actual}}$',fontdict={'fontsize':16})
axCV.tick_params(labelsize = 14)
plt.savefig('./DS4B-CO2-images/KFold-cv.png',dpi=300,bbox_inches='tight')
plt.show()

<div class="rqE">
Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample. It generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split. Although the CO<sub>2</sub> data set contains more than 10k samples, it is always worth proving the stability of an ML algorithm. As a result, it is a pity that a scheme such as the one just determined above (and in principle close to the one reported below) is not available in the Song <i>et al.</i> publication, at least in the supplementary information part.
<p style="text-align: center"><img width="650px" src="./DS4B-CO2-images/KFold-cvK.png" style="margin-left:auto; margin-right:auto" id="img_ResultsSong"></p>
<b>This error plot shows a good performance of the ML algorithm, as well as similar error bars, regardless of the training data set.</b>
</div>

<div class="rqE">

**Do you want to evaluate the last optimized model of the K-fold process?**

evaluate the next Python cell for compatibility purpose with the **Model Evaluation** section
</div>

In [ ]:
y_train = y_train_cv.copy()
y_test = y_valid_cv.copy()
ytest_hat = yvalid_hat.copy()

## Load and apply a previously saved model

### Load the database with unknown ILs

In [ ]:
dataCO2newf='DS4B-CO2-data'+'/'+'dataCO2-new.csv'
dataCO2new=pd.read_csv(dataCO2newf,sep=";",header=0)
display(dataCO2new)
inputANN = dataCO2new.drop(['IL','cation','anion','xCO2 ref'],axis=1)

<div class='rqE'>

The first ILs of this database have experimentally measured solubilities, whilst the major part have `NaN` unknown solubilities. They will be predicted for the temperature and pressure given in the table.
</div>

### Initial model of the reference paper

In [ ]:
from tensorflow.keras.models import load_model

# load models
BasicModel= load_model('./DS4B-CO2-SavedModels/ANNmodel-CO2-basic.keras')
# summarize model.
BasicModel.summary()

In [ ]:
dataCO2new_withPredictions = dataCO2new[['IL','cation','anion','xCO2 ref','T (K)','P (bar)']].copy()
xCO2newBasic = BasicModel.predict(inputANN)
dataCO2new_withPredictions.insert(loc = 4, column = 'xCO2 BM', value = xCO2newBasic)

pd.options.display.max_rows = len(dataCO2new_withPredictions)
display(dataCO2new_withPredictions)
pd.options.display.max_rows = 5

### Models 3, *i.e.* model 2 validated by *k*-fold cross validation

In [ ]:
dataCO2new_withPredictions = dataCO2new[['IL','cation','anion','xCO2 ref','T (K)','P (bar)']].copy()
dataCO2new_withPredictions.insert(loc = 4, column = 'xCO2 BM', value = xCO2newBasic)

from sklearn import preprocessing
from pickle import load as loadScaler

inputANNIL = inputANN.drop(['T (K)','P (bar)'],axis=1)
inputANNTP = inputANN[['T (K)','P (bar)']]

for i in range(5):
    #del nameModel
    scalerName = "./DS4B-CO2-SavedModels/scalerB-" + str(i) + ".pkl"
    scalerTP = loadScaler(open(scalerName, 'rb'))
    nameModelVar = f'ModelStzB{i}'
    fileName =  "./DS4B-CO2-SavedModels/ANNmodel-CO2-StandardizedB-" + str(i) + ".keras"
    exec("%s = '%s'" %("nameModel",nameModelVar))
    columnName = 'xCO2 StzB'+str(i)
    print(f"{bg.DARKREDB}ANN model {nameModelVar} (file = {fileName}). xCO2 will be stored in the {columnName} column{bg.OFF}")
    nameModel = load_model(fileName)
    nameModel.summary()
    
    inputANNTP_scaled = scalerTP.transform(inputANNTP.values) #returns a numpy array
    dfinputANNTP_scaled = pd.DataFrame(inputANNTP_scaled, index=inputANNTP.index, columns=inputANNTP.columns)
    inputANNB = pd.concat([dfinputANNTP_scaled,inputANNIL], axis=1)
    
    xCO2new = nameModel.predict(inputANNB)
    dataCO2new_withPredictions.insert(loc = 5+i, column = columnName, value = xCO2new)
    
pd.options.display.max_rows = len(dataCO2new_withPredictions)
display(dataCO2new_withPredictions)
pd.options.display.max_rows = 5

<a id="model-evaluation"></a>
## Model evaluation

### Loss, MAE and MSE as a function of epochs = during the ANN optimization with the back-propagation algorithm 

In [ ]:
ANNscoreTrain= ANNmodel.evaluate(x_train, y_train, verbose=0)
print(f"{color.GREEN}x_train / loss      : {ANNscoreTrain[0]:5.4f}")
print(f"x_train / mae       : {ANNscoreTrain[1]:5.4f}")
print(f"x_train / mse       : {ANNscoreTrain[2]:5.4f}{color.OFF}")
print()
ANNscoreEval = ANNmodel.evaluate(x_test, y_test, verbose=0)
print(f"{color.BLUE}x_test / loss       : {ANNscoreEval[0]:5.4f}")
print(f"x_test / mae        : {ANNscoreEval[1]:5.4f}")
print(f"x_test / mse        : {ANNscoreEval[2]:5.4f}{color.OFF}")

df=pd.DataFrame(data=ANNhistory.history)
plt.rcParams["figure.figsize"] = (14,6)
figMAE=df.plot(y=["mae","val_mae"],linestyle='-', marker='o',fontsize=14)
figMAE.set_xlabel('epoch',fontdict={'fontsize':16})
figMAE.set_ylabel('mae',fontdict={'fontsize':16})
figMAE.set_ylim([0.0,0.2])
figMAE.legend(loc='upper right', shadow=True, fontsize='14')
figMSE=df.plot(y=["mse","val_mse"],linestyle='-', marker='o',fontsize=14)
figMSE.set_xlabel('epoch',fontdict={'fontsize':16})
figMSE.set_ylabel('mse',fontdict={'fontsize':16})
figMSE.set_ylim([0.0,0.1])
figMSE.legend(loc='upper right', shadow=True, fontsize='14')

### Comparison between experimental and predicted CO<sub>2</sub> solubility

In [ ]:
print(y_train,ytrain_hat)

In [ ]:
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

xD=[0,1]
yD=[0,1]
fig = plt.figure(figsize=(10, 14))
plt.rcParams["font.size"] = (18) #font size
plt.tick_params(labelsize = 14)
gs = gridspec.GridSpec(2, 1, height_ratios=[10, 4], hspace=0, figure=fig)
ax0=fig.add_subplot(gs[0,0])
ax0.plot(y_train,ytrain_hat,'o',color="#1f77b4",label='training set')
ax0.plot(y_test,ytest_hat,'P',color="orange",label='validation set')
ax0.plot(xD,yD,'c',linestyle='--',lw=3)
ax0.set_xlabel('experimental CO$_2$ solubility',fontsize=16)
ax0.set_ylabel('predicted CO$_2$ solubility',fontsize=16)
ax0.set_xlim(0,1);ax0.set_ylim(0,1) 
ax0.legend(loc='upper left', shadow=True, fontsize='16')
plt.setp(ax0.get_xticklabels(), visible=False) #cosmetics

# ax1=fig.add_subplot(gs[1,0])
# ax1.plot(y_train,diffyt,'o',color="#1f77b4",label='training set')
# ax1.plot(y_test,diffyp,'P',color="orange",label='validation set')
# ax1.axhline(y=0, xmin=0., xmax=1.0, color='c', linestyle='--', linewidth=3)
# ax1.set_xlabel('experimental CO$_2$ solubility',fontsize=16)
# ax1.set_ylabel('residuals (exp./pred. error)',fontsize=16)
# ax1.set_xlim(0.0,1.0); ax1.set_ylim(-0.3,0.3)
# ax1.axes.get_yaxis().set_major_locator(MaxNLocator(prune='upper'))  #cosmetics
# gs.tight_layout(fig)
# plt.savefig('./DS4B-CO2-images/predictionCO2.png',dpi=300,bbox_inches='tight')
plt.show()


### Residuals plotted as histograms (distribution of the error)

In [ ]:
# numpy.ravel() returns a contiguous flattened array
print(color.BLUE,
      "Train.","mean: {:.3f}".format(np.mean(diffyt)),
      "   std: {:.3f}".format(np.std(diffyt)),
      "   MAE: {:.3f}".format(np.average(abs(diffyt))),
      "    R2: {:.3f}".format(np.corrcoef(y_train.to_numpy(),ytrain_hat.flatten())[0,1])
     )
print(color.YELLOW,
      " Test.","mean: {:.3f}".format(np.mean(diffyp)),
      "   std: {:.3f}".format(np.std(diffyp)),
      "   MAE: {:.3f}".format(np.average(abs(diffyp))),
      "    R2: {:.3f}".format(np.corrcoef(y_test.to_numpy(),ytest_hat.flatten())[0,1])
     )

# make a dataframe
df1 = pd.DataFrame(
    {
        "Error distribution": diffyp,
        "Type": 'Test set'    },
)
#display(df1)
df2 = pd.DataFrame(
    {
        "Error distribution": diffyt,
        "Type": 'Training set'    },
)
#display(df2)
distrib=pd.concat([df1,df2],ignore_index=True)
#display(distrib)

xlim=0.2
nbins=120

import matplotlib.gridspec as gridspec
plt.rcParams["figure.figsize"] = (8,13) #graphic size
plt.rcParams["font.size"] = (18) #font size
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
plt.subplot(gs[0])
fig1=sns.histplot(data=distrib,x="Error distribution", hue="Type", bins=nbins, kde=True, palette=['Orange','#1f77b4'], alpha=1.0)
plt.xlabel("$x_{\mathrm{CO_2}}^{\mathrm{pred}}-x_{\mathrm{CO_2}}^{\mathrm{exp}}$")
plt.ylabel("number of data points in each interval")
plt.xlim(-xlim,xlim)
plt.subplot(gs[1])
plt.xlim(-xlim,xlim)
fig2=sns.boxplot(data=distrib, x="Error distribution", hue="Type", notch=True, orient = 'h',palette=['Orange','#1f77b4'],legend=False)
fig2.set(ylabel=None)
plt.savefig('./DS4B-CO2-images/errorCO2.png',dpi=300,bbox_inches='tight')

### Identify the most significant failures?

#### Save y_hat of the training and of the test set in the dataCO2 dataframe

In [ ]:
display(dataCO2)
df_ytrain_hat = pd.DataFrame(ytrain_hat,index=x_train.index,columns=["x_CO2_hat"])
df_ytrain_hat["set"] = "train"
#display(df_ytrain_hat)
df_ytest_hat = pd.DataFrame(ytest_hat,index=x_test.index,columns=["x_CO2_hat_tss"])
df_ytest_hat["set_tss"] = "test"
#display(df_ytest_hat)
dataCO2_hat = pd.concat([dataCO2,df_ytrain_hat,df_ytest_hat],axis=1)
dataCO2_hat['x_CO2_hat'].update(dataCO2_hat.pop('x_CO2_hat_tss'))
dataCO2_hat['set'].update(dataCO2_hat.pop('set_tss'))
display(dataCO2_hat)

#### Calculate residuals and relative errors

<div class="intro">
    
They will be stored in a new `dataCO2_hat` dataframe
</div>

In [ ]:
dataCO2_hat["residual"] = dataCO2_hat.loc[:,["x_CO2","x_CO2_hat"]].apply(lambda x: x.iloc[0]-x.iloc[1], axis=1)
dataCO2_hat["relative err."] = dataCO2_hat.loc[:,["x_CO2","x_CO2_hat"]].apply(lambda x: round(100*abs(x.iloc[0]-x.iloc[1])/x.iloc[0],1), axis=1)
move = "x_CO2_hat"
column_to_move = dataCO2_hat.pop(move)
dataCO2_hat.insert(6, move, column_to_move)
move = "residual"
column_to_move = dataCO2_hat.pop(move)
dataCO2_hat.insert(7, move, column_to_move)
move = "relative err."
column_to_move = dataCO2_hat.pop(move)
dataCO2_hat.insert(8, move, column_to_move)
move = "set"
column_to_move = dataCO2_hat.pop(move)
dataCO2_hat.insert(9, move, column_to_move)
display(dataCO2_hat)

#### Highlight the most significant errors

In [ ]:
threshold = 0.20
dataCO2_Large_Error = dataCO2_hat.query("abs(residual) >= @threshold")
display(dataCO2_Large_Error)
dataCO2_Large_Error.to_csv("./DS4B-CO2-data/dataCO2_withPrediction_Wrong.csv")
dataCO2_Small_Error = dataCO2_hat.query("abs(residual) < @threshold")
display(dataCO2_Small_Error)
dataCO2_Small_Error.to_csv("./DS4B-CO2-data/dataCO2_withPrediction_Right.csv")

#### Plot x_CO2 & x_CO2_hat = f(T,P) for a given IL (bogus part...)

##### Wrong predictions

In [ ]:
# import plotly.graph_objects as go 
# import plotly.offline as off
# from scipy.interpolate import griddata

# off.init_notebook_mode()

# TN = 'T (K)'
# PN = 'P (bar)'
# xCO2N = 'x_CO2'
# xCO2hatN = 'x_CO2_hat'
# #### Choose an IL that belongs to the dataCO2_Large_Error dataframe
# ILN = '[HMIM][BF4]'

# #Create meshgrid for T & P
# Ti = np.linspace(min(dataCO2_hat.query("IL == @ILN")[TN]), max(dataCO2_hat.query("IL == @ILN")[TN]), num=100)
# Pi = np.linspace(min(dataCO2_hat.query("IL == @ILN")[PN]), max(dataCO2_hat.query("IL == @ILN")[PN]), num=100)

# T_grid, P_grid = np.meshgrid(Ti,Pi)

# # Grid data
# xCO2_grid = griddata((dataCO2_hat.query("IL == @ILN")[TN],dataCO2_hat.query("IL == @ILN")[PN]),dataCO2_hat.query("IL == @ILN")[xCO2N],(T_grid,P_grid),method='cubic')
# xCO2_hat_grid = griddata((dataCO2_hat.query("IL == @ILN")[TN],dataCO2_hat.query("IL == @ILN")[PN]),dataCO2_hat.query("IL == @ILN")[xCO2hatN],(T_grid,P_grid),method='cubic')


# # Plotly 3D Surface
# fig_ref = go.Surface(x=T_grid,y=P_grid,z=xCO2_grid,
#                        colorscale='viridis')
# fig_hat = go.Surface(x=T_grid,y=P_grid,z=xCO2_hat_grid,
#                        colorscale='greys')

# layout = go.Layout(title='CO$_2$ solubility', autosize=False,
#                   width=700, height=700,
#                   margin=dict(l=65, r=50, b=65, t=90),
#                   scene=dict(
#                             xaxis_title=TN,
#                             yaxis_title=PN,
#                             zaxis_title=xCO2N,
#     ),
#                  )
# off.iplot({"data": [fig_ref, fig_hat],"layout": layout})


##### Good predictions

In [ ]:
# import plotly.graph_objects as go 
# import plotly.offline as off
# from scipy.interpolate import griddata

# off.init_notebook_mode()

# TN = 'T (K)'
# PN = 'P (bar)'
# xCO2N = 'x_CO2'
# xCO2hatN = 'x_CO2_hat'
# #### Choose an IL that belongs to the dataCO2_Small_Error dataframe
# ILN = '[BMIM][BF4]'

# #Create meshgrid for T & P
# Ti = np.linspace(min(dataCO2_hat.query("IL == @ILN")[TN]), max(dataCO2_hat.query("IL == @ILN")[TN]), num=100)
# Pi = np.linspace(min(dataCO2_hat.query("IL == @ILN")[PN]), max(dataCO2_hat.query("IL == @ILN")[PN]), num=100)

# T_grid, P_grid = np.meshgrid(Ti,Pi)

# # Grid data
# xCO2_grid = griddata((dataCO2_hat.query("IL == @ILN")[TN],dataCO2_hat.query("IL == @ILN")[PN]),dataCO2_hat.query("IL == @ILN")[xCO2N],(T_grid,P_grid),method='cubic')
# xCO2_hat_grid = griddata((dataCO2_hat.query("IL == @ILN")[TN],dataCO2_hat.query("IL == @ILN")[PN]),dataCO2_hat.query("IL == @ILN")[xCO2hatN],(T_grid,P_grid),method='cubic')

# # Plotly 3D Surface
# fig_ref = go.Surface(x=T_grid,y=P_grid,z=xCO2_grid,
#                        colorscale='viridis')
# fig_hat = go.Surface(x=T_grid,y=P_grid,z=xCO2_hat_grid,
#                        colorscale='greys')

# layout = go.Layout(title='CO$_2$ solubility', autosize=False,
#                   width=700, height=700,
#                   margin=dict(l=65, r=50, b=65, t=90),
#                   scene=dict(
#                             xaxis_title=TN,
#                             yaxis_title=PN,
#                             zaxis_title=xCO2N,
#     ),
#                  )
# off.iplot({"data": [fig_ref, fig_hat],"layout": layout})


## "Simple" exercise

<div class="exE" title="Optimize the hyperparameters of Model 2 by hand">

Return to the <a href="#model2">corresponding section</a> and change some hyperparameters. Carefully note the accuracy of each attempt, and watch out for a possible overfitting. Suggestions:
- Try to increase the number of neurons of the hidden layer
- Replace the `tanh` activation function with the `relu` activation function
- Add a second hidden layer (this is **deep learning**!)

Do you find a better model than the original model 2?
</div>

## Appendix: activation functions

In [ ]:
def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)

z = np.linspace(-5, 5, 200)
pw=4
ph=4


### relu function

$$R(z)=\begin{cases}
\begin{array}{cc}
z & \,\,\,\,\,\,\,z>0\\
0 & \,\,\,\,\,\,\,z\leq0
\end{array}\end{cases}$$

In [ ]:
def relu(z):
    return np.maximum(0, z)

figR, axR = plt.subplots(1, 1)
figR.set_size_inches(pw,ph)
axR.axhline(y=0, linewidth=2, linestyle='--', color='lightgray')
axR.axvline(x=0, linewidth=2, linestyle='--', color='lightgray')
axR.plot(z, relu(z),linewidth=4,label="ReLU")
axR.plot(z, derivative(relu, z), linewidth=4, alpha=0.6, label="dReLU/dx")
axR.legend(loc='upper left', shadow=True, fontsize='14')
axR.set_xlabel('x',fontdict={'fontsize':16})
axR.set_ylabel('f(x)',fontdict={'fontsize':16})
axR.set_xlim(-5, 5)
axR.set_ylim(-0.5, 7)
axR.tick_params(labelsize = 14)
plt.show()


### tanh function

In [ ]:
def tanh(z):
    return np.tanh(z)

figT, axT = plt.subplots(1, 1)
figT.set_size_inches(pw,ph)
axT.axhline(y=0, linewidth=2, linestyle='--', color='lightgray')
axT.axvline(x=0, linewidth=2, linestyle='--', color='lightgray')
axT.plot(z, tanh(z),linewidth=4,label="tanh")
axT.plot(z, derivative(tanh, z), linewidth=4, alpha=0.6, label="dtanh/dx")
axT.legend(loc='upper left', shadow=True, fontsize='14')
axT.set_xlabel('x',fontdict={'fontsize':16})
axT.set_ylabel('f(x)',fontdict={'fontsize':16})
axT.set_xlim(-5, 5)
axT.set_ylim(-2, 2)
axT.tick_params(labelsize = 14)
plt.show()


### softmax function
$$\sigma_{i}(\boldsymbol{x})=\frac{e^{x_{i}}}{\sum_{j=1}^{K}e^{x_{j}}}$$

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x)) #this implentation has a better numerical stability
    return e_x / e_x.sum()

z2 = [-2., -1., -0.5, 0., 0.5, 1., 2.]
figS, axS = plt.subplots(1, 1)
figS.set_size_inches(pw,ph)
axS.axhline(y=0, linewidth=2, linestyle='--', color='lightgray')
axS.axvline(x=0, linewidth=2, linestyle='--', color='lightgray')
axS.plot(z2, softmax(z2),linewidth=4,marker='o',label="softmax")
axS.legend(loc='upper left', shadow=True, fontsize='14')
axS.set_xlabel('x',fontdict={'fontsize':16})
axS.set_ylabel('f(x)',fontdict={'fontsize':16})
axS.set_xlim(-2.1, 2.1)
axS.set_ylim(0., 1.0)
axS.tick_params(labelsize = 14)
plt.show()
print(f"Sum of softmax values: {softmax(z2).sum()}")

In [ ]:
vID.end(cwd0)

# END